In [1]:
# Modules
import os
import csv
import pprint
import pandas as pd
import glob
import pprint
import datetime
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
!pwd

/Users/youngyou/Desktop/baseball-predictor/young


In [2]:
# Support functions
def date_str(date: int):
    s = str(date)
    return f"{s[0:4]}-{s[4:6]}-{s[6:]}"


# Passed the gamedays series (YYYYMMDD) and season (year) return with the first
# and last game dates.
def date_range(gamedays, season):
    # Converts 'YYYY' string to an integer start date, YYYY0101 and end date YYYY1231
    def season_to_date(season): 
#         return (int(season)*10000 + 101, int(season)*10000+1231)
        # Hack to shorten the season for dev purposes (faster analysis)
        return (int(season)*10000 + 101, int(season)*10000+430)
    
    first_of_year, last_of_year = season_to_date(season)
    season_gamedays = gamedays.loc[((gamedays >= first_of_year) &
                                    (gamedays <= last_of_year))]
    return (season_gamedays.iloc[0], season_gamedays.iloc[-1])
    

# returns new game date offset by n. Passed in the gamedays series.
def gamedays_offset(gamedays, base_date, n): 
    if base_date not in gamedays.values:
        raise ValueError(f"{base_date}, not in the the gamedays series.")
    base_date_index = gamedays[gamedays==base_date].index[0]
    if ((n + base_date_index) < 0) or n + base_date_index >= len(gamedays):
        raise ValueError(f"Attempting to calculate a game date outside the dataset.")
        return(0)  # Out of range
    else:
        new_index = base_date_index + n
        return gamedays.iloc[new_index]
    
def derive_metrics(results_df):
    num_games = len(results_df)
    num_correct = results_df['Prediction Correct?'].values.sum()
    percent_correct = num_correct/num_games*100.
    return (num_games, num_correct, percent_correct)

In [9]:
def read_source_data():
    def reader(f):
        df = pd.read_csv(f, index_col=False, header=None)  
        df.columns = [("Col_"+str(i)) for i in range(1,df.shape[1]+1)]       
        return df

    files = glob.glob("../datasets/Final_Data_Files/GL*.csv")
    files.sort()
    df = pd.concat([reader(f) for f in files])
    old_df_len = len(df)
    
    # Insert column headers
df = df.rename(columns={'Col_1':'Date',
                            'Col_4':'Visiting Team',
                            'Col_5':'Visiting League',
                            'Col_7':'Home Team',
                            'Col_8':'Home League',
                            'Col_10':'Visiting Score',
                            'Col_11':'Home Score',
                            'Col_26':'Visiting Homerun',
                            'Col_54':'Home Homerun'
                           }).reset_index()

df = df[['Date', 'Visiting Team', 'Visiting League', 'Home Team', 'Home League',
            'Visiting Score','Home Score', 'Visiting Homerun ','Home Homerun']]
df = df.replace('FLO','MIA') # After the 2011 season, the Florida Marlins
#                                  # rebranded themselves the Miami Marlins. This
#                                  # search and replace makes the two the same.
Drop all rows with missing information
print(df.head()  
      #df = df.dropna(how='any')
 if len(df) < old_df_len:
#         print(f"Dropped {old_df_len-len(df)} rows due to missing data.")
    
#     # Create new columns we'll need. 
# #     df['Key'] = df['Date'].astype(str) + df['Visiting Team']
#     df['Home Winner'] = df['Home Score'] > df['Visiting Score']
#     df['V NetHomeruns'] = df['Visiting Homerun'] - df['Home Homerun']
#     df['H NetHomeruns'] = - df['V NetHomeruns']
    
#     #     print(df.shape)
#     print(f"Dataset loaded with {df.shape[0]} games, ", end='')
#     print(f"{df.shape[1]} columns, {date_str(df.iloc[0, 0])} - ", end='')
#     print(f"{date_str(df.iloc[-1,0])}")
#     return df

# read_source_data()

SyntaxError: invalid syntax (<ipython-input-9-89eafe29cafa>, line 29)

In [5]:
    
    df = df.replace('FLO','MIA') # After the 2011 season, the Florida Marlins
                                 # rebranded themselves the Miami Marlins. This
                                 # search and replace makes the two the same.
    
    # Drop all rows with missing information
    # print(df.head())
    df = df.dropna(how='any')
    if len(df) < old_df_len:
        print(f"Dropped {old_df_len-len(df)} rows due to missing data.")
    
    # Create new columns we'll need. 
#     df['Key'] = df['Date'].astype(str) + df['Visiting Team']
    df['Home Winner'] = df['Home Score'] > df['Visiting Score']
    df['V NetHomeruns'] = df['Visiting Homerun'] - df['Home Homerun']
    df['H NetHomeruns'] = - df['V NetHomeruns']
    
    #     print(df.shape)
    print(f"Dataset loaded with {df.shape[0]} games, ", end='')
    print(f"{df.shape[1]} columns, {date_str(df.iloc[0, 0])} - ", end='')
    print(f"{date_str(df.iloc[-1,0])}")
    return df

read_source_data()

NameError: name 'df' is not defined

In [ ]:
# Given a games df with predictions and prediction resuls by game already captured,
# generate an outcomes dataframe with stats by season.
def append_outcomes(outcomes_df, g, ra_d): # g is the games dataframe with predictions
                                             # d is the dictionary with run attributes to log
    # 1. Capture results spanning the entire dataset (all seasons)
    num_games = len(g)
    num_correct = g['Prediction Correct?'].values.sum()
    home_win_percent = g['Home Winner'].values.sum() / num_games * 100
    home_predict_percent = g['Home Wins?'].values.sum() / num_games * 100
    percent_correct = num_correct/num_games*100.
    
    new_row = pd.Series({'Predictor':'Net or Raw Points (see Net)',
                         'Season':0, # Indicates all seasons in the dataset.
                         'Lookback':ra_d['Lookback'],
                         'Net':ra_d['Net'],
                         'Num Games':num_games,
                         'Home Win %':home_win_percent,
                         'Home Predict %':home_predict_percent,
                         'Num Correct':num_correct,
                         '% Correct':percent_correct,
                         'Comment':('All Seasons' + ra_d['Comment'])})
    outcomes_df = outcomes_df.append(new_row, ignore_index=True)
    
    # 2. Caculate individual seasons represented in the dataset
    seasons = g['Date'].transform(lambda x: int(x/10000)).unique().tolist()
#     pprint.pprint(seasons)
    
    for season in seasons:
        start_date = int(season)*10000 + 101  # YYYYMMDD, Jan 01 of the Year
        end_date = int(season)*10000 + 1231   # YYYYMMDD, DEC 31 of the Year
        season_df = g.loc[((g['Date']>= start_date) &(g['Date']<= end_date)), :]
        
        num_games = len(season_df)
        num_correct = season_df['Prediction Correct?'].values.sum()
        home_win_percent = season_df['Home Winner'].values.sum() / num_games * 100
        home_predict_percent = season_df['Home Wins?'].values.sum() / num_games * 100
        percent_correct = num_correct/num_games*100.
        
        new_row = pd.Series({'Predictor':ra_d['Predictor'],
                     'Season':season,
                     'Lookback':ra_d['Lookback'],
                     'Net':ra_d['Net'],
                     'Num Games':num_games,
                     'Home Win %':home_win_percent,
                     'Home Predict %':home_predict_percent,
                     'Num Correct':num_correct,
                     '% Correct':percent_correct,
                     'Comment':('All Seasons' + ra_d['Comment'])})
        
        outcomes_df = outcomes_df.append(new_row, ignore_index=True)
        
    return outcomes_df